##**Instaladores**

In [ ]:
!pip install pandas scikit-learn matplotlib seaborn >nul 2>&1
!pip install catboost >nul 2>&1

In [ ]:
!pip install optuna >nul 2>&1

In [ ]:
!pip install -U lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


##**Librerias**

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,cohen_kappa_score, mean_absolute_error
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.base import BaseEstimator, ClassifierMixin
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

drive.mount('/content/drive')
pd.set_option('display.max_columns', None)

Mounted at /content/drive


##**Funciones**

In [ ]:
#Tabla resumen
def resumen(dataframe):
  # Cálculos de resumen
  null_counts = dataframe.isnull().sum()
  total_counts = len(dataframe)
  non_null_counts = dataframe.notnull().sum()
  data_types = dataframe.dtypes
  unique_counts = dataframe.nunique()
  # Crear DataFrame de resumen
  summary_df = pd.DataFrame({
      'Tipo de Dato': data_types,
      'Valores Únicos': unique_counts,
      'No Nulos': non_null_counts,
      'Nulos': null_counts,
      '% Nulos': (null_counts / total_counts) * 100
  })
  # Ordenar por % de nulos
  summary_df = summary_df.sort_values(by='% Nulos', ascending=False)
  # Aplicar formato con Styler
  styled_summary = summary_df.style \
      .format({'% Nulos': '{:.2f}%'}) \
      .bar(subset='% Nulos', color='#f08080') \
      .set_properties(**{'text-align': 'center'}) \
      .set_caption("Resumen de Columnas del DataFrame") \
      .set_table_styles([
          # Color del encabezado de columnas
          {'selector': 'th', 'props': [
              ('text-align', 'center'),
              ('background-color', '#1f4e79'),
              ('color', 'white'),
              ('font-weight', 'bold')
          ]},
          # Color de las celdas
          {'selector': 'td', 'props': [('padding', '5px')]},
          # Estilo del título de la tabla
          {'selector': 'caption', 'props': [
              ('color', '#1f4e79'),
              ('font-size', '18px'),
              ('text-align', 'center'),
              ('font-weight', 'bold')
          ]}
      ])
  # Mostrar
  display(styled_summary)
  # Información final
  print(f"\n🧾 El DataFrame tiene un total de **{len(dataframe.columns)}** columnas y **{len(dataframe)}** filas.")

In [ ]:
#Función Encabezado
def encabezado(dataframe,filas=5):
  salida=dataframe.head(filas).style \
      .set_properties(**{
          'background-color': '#f9f9f9',
          'border': '1px solid #ddd',
          'text-align': 'center'
      }) \
      .set_table_styles([
          {'selector': 'th', 'props': [('background-color', '#4B8BBE'), ('color', 'white'), ('text-align', 'center')]}
      ]) \
      .highlight_null() \
      .set_caption("🔍 Vista Previa de los Primeros Registros")
  display(salida)

##**Cargar Datos**

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/X_train.csv')
Y_train = pd.read_csv('/content/drive/MyDrive/y_train.csv')
X_test = pd.read_csv('/content/drive/MyDrive/X_test.csv')
id_test = pd.read_csv('/content/drive/MyDrive/id_test.csv')

In [ ]:
resumen(X_train)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
ESTU_PRGM_DEPARTAMENTO,float64,688698,692500,0,0.00%
ESTU_VALORMATRICULAUNIVERSIDAD,float64,692330,692500,0,0.00%
ESTU_HORASSEMANATRABAJA,float64,692430,692500,0,0.00%
FAMI_ESTRATOVIVIENDA,float64,692459,692500,0,0.00%
FAMI_TIENEINTERNET,int64,2,692500,0,0.00%
FAMI_EDUCACIONPADRE,float64,692062,692500,0,0.00%
FAMI_TIENELAVADORA,int64,2,692500,0,0.00%
FAMI_TIENEAUTOMOVIL,int64,2,692500,0,0.00%
ESTU_PAGOMATRICULAPROPIO,int64,2,692500,0,0.00%
FAMI_TIENECOMPUTADOR,int64,2,692500,0,0.00%



🧾 El DataFrame tiene un total de **25** columnas y **692500** filas.


In [ ]:
encabezado(X_train)

,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4,EDUMADRE_DISCRIMINANTE,EDUPADRE_DISCRIMINANTE,GRUPO_DEPARTAMENTO,ISE,ESFUERZO_FAMILIAR,ACCESO_TECNOLOGICO,NIVEL_SOCIOFAMILIAR,DESAJUSTE_ECONOMICO,INCONGRUENCIA_ISE_MATRICULA,ESFUERZO_NORM
0,1.491769,1.491769,1.491769,1.491769,1,1.491769,1,1,0,1,1.491769,0.437002,-0.556223,0.813978,0.060296,1.491769,1.491769,1.491769,1.386568,0.320712,0.415248,1.121436,1.150788,-1.193493,-0.649471
1,1.491769,1.491769,1.491769,1.745884,0,1.491769,1,0,0,1,1.491769,0.346934,-0.481341,0.508180,0.016142,1.491769,1.491769,1.491769,0.374613,-1.022448,-0.761210,0.412067,0.012376,-0.355092,-0.933019
2,1.745884,1.245884,1.491769,1.497256,1,1.491769,1,0,0,0,1.491769,0.232301,-0.492038,0.729034,0.016142,1.491769,1.245884,1.745884,-0.059082,0.768431,-1.937669,-0.119960,0.012376,0.147948,0.276783
3,1.491769,1.491769,1.245884,1.491769,1,1.491769,1,0,0,1,1.245884,1.583325,-0.941332,-0.171371,-0.690324,1.245884,1.745884,1.497256,-0.348212,-0.574728,0.415248,-0.651987,0.012376,0.650989,-0.351747
4,1.491769,1.163923,1.491769,1.372942,1,1.491769,1,1,0,1,1.491769,0.387874,-0.299484,0.389259,0.457683,1.491769,1.491769,1.122942,-0.348212,0.320712,0.415248,-0.651987,0.012376,0.483309,0.243702


In [ ]:
resumen(X_test)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
ESTU_PRGM_DEPARTAMENTO,float64,31,296786,0,0.00%
ESTU_VALORMATRICULAUNIVERSIDAD,float64,8,296786,0,0.00%
ESTU_HORASSEMANATRABAJA,float64,5,296786,0,0.00%
FAMI_ESTRATOVIVIENDA,float64,7,296786,0,0.00%
FAMI_TIENEINTERNET,int64,2,296786,0,0.00%
FAMI_EDUCACIONPADRE,float64,12,296786,0,0.00%
FAMI_TIENELAVADORA,int64,2,296786,0,0.00%
FAMI_TIENEAUTOMOVIL,int64,2,296786,0,0.00%
ESTU_PAGOMATRICULAPROPIO,int64,2,296786,0,0.00%
FAMI_TIENECOMPUTADOR,int64,2,296786,0,0.00%



🧾 El DataFrame tiene un total de **25** columnas y **296786** filas.


In [ ]:
encabezado(X_test)

,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,coef_1,coef_2,coef_3,coef_4,EDUMADRE_DISCRIMINANTE,EDUPADRE_DISCRIMINANTE,GRUPO_DEPARTAMENTO,ACCESO_TECNOLOGICO,ESFUERZO_FAMILIAR,ISE,NIVEL_SOCIOFAMILIAR,DESAJUSTE_ECONOMICO,INCONGRUENCIA_ISE_MATRICULA,ESFUERZO_NORM
0,1.576757,1.391997,1.396907,1.482493,1,1.452718,1,0,1,1,1.674478,0.492496,-0.433304,0.907947,-0.228590,1.674478,1.517607,1.579579,0.366753,-1.462608,-0.381257,0.027996,-1.544018,0.009925,-0.959924
1,1.407371,1.678098,1.545870,1.609555,1,1.548897,1,0,0,1,1.450326,-0.339797,0.243260,0.555805,0.910679,1.523793,1.517607,1.436167,0.366753,0.319867,0.208465,0.208165,0.604367,-0.161679,-0.140174
2,1.458951,1.684645,1.245612,1.482493,1,1.609625,1,0,0,1,1.569309,0.138153,-0.338162,0.522267,-0.228590,1.523793,1.519139,1.436167,0.366753,-1.462608,-0.233826,-0.152174,-0.469826,0.181529,-1.015564
3,1.716829,1.684645,1.545870,1.668387,0,1.688510,1,0,0,0,1.695980,-0.891912,1.564675,-0.735383,0.466808,1.687699,1.683031,1.579579,-3.721459,-0.125752,-1.560699,-1.773695,0.604367,1.725965,1.877672
4,1.481302,1.678098,1.492219,1.609555,1,1.548897,1,0,0,1,1.569309,-0.488126,0.243260,0.723491,0.348442,1.523793,1.517607,1.579579,0.366753,-0.125752,-0.086396,-0.152174,0.604367,0.181529,-0.224251


In [ ]:
encabezado(Y_train)

,RENDIMIENTO_GLOBAL
0,medio-alto
1,bajo
2,bajo
3,alto
4,medio-bajo


In [ ]:
#Codificacion Ordinal, Importante para el modelo
ordinal_Y={
    'alto':3,
    'medio-alto':2,
    'medio-bajo':1,
    'bajo':0
}
Y_train['RENDIMIENTO_GLOBAL']=Y_train['RENDIMIENTO_GLOBAL'].map(ordinal_Y)

In [ ]:
encabezado(Y_train)

,RENDIMIENTO_GLOBAL
0,2
1,0
2,0
3,3
4,1


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train)

##**Modelos**

In [ ]:
# ⚙️ Mejores hiperparámetros (Optuna)
best_params = {
    'objective': 'multiclass',             # Tipo de problema multiclase
    'num_class': 4,                        # Número de clases a predecir
    'learning_rate': 0.02085,              # Tasa de aprendizaje óptima
    'num_leaves': 119,                     # N° máximo de hojas por árbol
    'max_depth': 10,                       # Profundidad máxima de cada árbol
    'feature_fraction': 0.7998,            # Porcentaje de características a usar por iteración
    'bagging_fraction': 0.8863,            # Fracción de datos usada en cada iteración (filas)
    'bagging_freq': 7,                     # Frecuencia de bagging
    'lambda_l1': 1.2673,                   # Regularización L1
    'lambda_l2': 4.0582,                   # Regularización L2
    'random_state': 42                     # Semilla para reproducibilidad
}


# 🔧 Modelos base

model_lgb = LGBMClassifier(**best_params)   # LightGBM con los mejores hiperparámetros

model_cat = CatBoostClassifier(
    verbose=0,              # No mostrar logs durante el entrenamiento
    random_state=42,        # Semilla para reproducibilidad
    task_type="CPU",        # ✅ CatBoost en CPU para evitar conflicto con GPU
    iterations=500          # Número de iteraciones (árboles)
)

model_xgb = XGBClassifier(
    tree_method="gpu_hist",       # ✅ Usa GPU con histograma para acelerar
    predictor="gpu_predictor",    # Predicción en GPU
    objective="multi:softmax",    # Multiclase con salida de clase (no probabilidades)
    num_class=4,                  # Número de clases
    eval_metric="mlogloss",      # Métrica de evaluación para el entrenamiento
    use_label_encoder=False,     # No usar codificador de etiquetas interno de XGBoost
    random_state=42
)


# 🧠 Meta-modelo
meta_model = LogisticRegression(max_iter=1000)  # Clasificador final (usa las predicciones de los modelos base)


# 🔀 Stacking
stacking = StackingClassifier(
    estimators=[
        ("lgb", model_lgb),     # Primer modelo base
        ("cat", model_cat),     # Segundo modelo base
        ("xgb", model_xgb)      # Tercer modelo base
    ],
    final_estimator=meta_model,  # Meta-modelo que aprende de las predicciones de los anteriores
    passthrough=True,            # 🔁 Pasa también las features originales al meta-modelo
    n_jobs=1                     # ✅ Entrena en serie (evita conflictos de GPU)
)

# 🏋️ Entrenamiento del modelo de stacking
stacking.fit(x_train, y_train.values.ravel())  # Entrena todos los modelos + meta-modelo

# 🔮 Predicción
y_pred = stacking.predict(x_test)

# 📊 Evaluación
acc = accuracy_score(y_test, y_pred)
qwk = cohen_kappa_score(y_test, y_pred, weights="quadratic")
mae = mean_absolute_error(y_test, y_pred)

print(f"📌 Accuracy: {acc:.4f}")
print(f"📌 QWK: {qwk:.4f}")
print(f"📌 MAE: {mae:.4f}")
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, digits=3))




[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.7998852890198828, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7998852890198828
[LightGBM] [Warning] lambda_l2 is set=4.0581878471785355, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.0581878471785355
[LightGBM] [Warning] lambda_l1 is set=1.2673292348186478, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2673292348186478
[LightGBM] [Warning] bagging_fraction is set=0.8863144821482485, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8863144821482485
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.7998852890198828, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7998852890198828
[LightGBM] [Warning] lambda_l2 is set=4.0581878471785355, reg_

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:23:10] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.7998852890198828, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7998852890198828
[LightGBM] [Warning] lambda_l2 is set=4.0581878471785355, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.0581878471785355
[LightGBM] [Warning] lambda_l1 is set=1.2673292348186478, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2673292348186478
[LightGBM] [Warning] bagging_fraction is set=0.8863144821482485, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8863144821482485
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.7998852890198828, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7998852890198828
[LightGBM] [Warning] lambda_l2 is set=4.0581878471785355, reg_

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:41:10] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:41:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:41:12] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:41:12] WARNING: /workspac

[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.7998852890198828, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7998852890198828
[LightGBM] [Warning] lambda_l2 is set=4.0581878471785355, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.0581878471785355
[LightGBM] [Warning] lambda_l1 is set=1.2673292348186478, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2673292348186478
[LightGBM] [Warning] bagging_fraction is set=0.8863144821482485, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8863144821482485


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:43:10] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


📌 Accuracy: 0.4215
📌 QWK: 0.4684
📌 MAE: 0.8195

📊 Classification Report:
              precision    recall  f1-score   support

           0      0.451     0.514     0.480     34597
           1      0.329     0.278     0.302     34455
           2      0.329     0.315     0.322     34324
           3      0.544     0.575     0.559     35124

    accuracy                          0.421    138500
   macro avg      0.413     0.421     0.416    138500
weighted avg      0.414     0.421     0.417    138500



In [ ]:
# ⚠️ Asegura que X_test tenga las mismas columnas (en el mismo orden) que X_train
X_test = x_test[X_train.columns]
# 🔮 Realiza la predicción sobre el conjunto de test usando el modelo entrenado (puede ser el stacking o un solo modelo)
Y_pred_test = model.predict(X_test)
# 🗃️ Convierte las predicciones a un DataFrame para su exportación a Kaggle
Y_pred_testd = pd.DataFrame(Y_pred_test, columns=['RENDIMIENTO_GLOBAL'])

[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.7998852890198828, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7998852890198828
[LightGBM] [Warning] lambda_l2 is set=4.0581878471785355, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.0581878471785355
[LightGBM] [Warning] lambda_l1 is set=1.2673292348186478, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2673292348186478
[LightGBM] [Warning] bagging_fraction is set=0.8863144821482485, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8863144821482485


In [ ]:
encabezado(Y_pred_testd)

,RENDIMIENTO_GLOBAL
0,0
1,2
2,3
3,0
4,0


In [ ]:
ordinal_Y={
    0:'bajo',
    1:'medio-bajo',
    2:'medio-alto',
    3:'alto'
}
Y_pred_testd['RENDIMIENTO_GLOBAL'] = Y_pred_testd['RENDIMIENTO_GLOBAL'].map(ordinal_Y)

In [ ]:
encabezado(Y_pred_testd)

,RENDIMIENTO_GLOBAL
0,bajo
1,medio-alto
2,alto
3,bajo
4,bajo


In [ ]:
id_test['RENDIMIENTO_GLOBAL']=Y_pred_testd['RENDIMIENTO_GLOBAL']
encabezado(id_test)

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


In [ ]:
id_test.to_csv('id_test_pred_STACKV2.csv', index=False)